In [15]:
import os

os.environ['SHELL']


'/bin/bash'

In [26]:
%%bash
brew install wget
wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py

==> Auto-updated Homebrew!
Updated 3 taps (heroku/brew, homebrew/core and homebrew/cask).
==> New Formulae
abseil
adios2
airshare
aliyun-cli
allureofthestars
alp
amp
anime-downloader
ansible@2.8
antibody
apollo-cli
appium
appstream-glib
archiver
arduino-cli
asimov
asuka
asymptote
asyncplusplus
atasm
atlantis
austin
autodiff
aws-cdk
aws-google-auth
awscli@1
awscurl
awsume
awsweeper
azcopy
b3sum
bandwhich
base91
bazelisk
bcoin
beancount
benthos
berglas
bic
bingrep
biosig
bitwise
blis
blogc
bnfc
bombadillo
bootloadhid
borgmatic
boringtun
breezy
broot
buildkit
buildozer
cacli
calceph
cargo-c
cargo-instruments
carton
cartridge-cli
cassandra-cpp-driver
cassowary
catch2
cbmc
ccfits
ccheck
ccls
cctz
cddlib
cdk8s
cdo
cedille
celero
cf-tool
cfn-lint
cgns
chezmoi
choose-rust
chrony
cjson
claws-mail
clip
clojure-lsp
cloud-nuke
cloudformation-cli
clp
clzip
cmix
cmt
cobalt
coconut
code-server
codespell
coinutils
colfer
comby
confd
container-structure-test
contentful-cli
coredns
corral
cpio
cpl
cpm
c

Updating Homebrew...
==> Downloading https://homebrew.bintray.com/bottles-portable-ruby/portable-ruby-2.6.3_2.yosemite.bottle.tar.gz
######################################################################## 100.0%
==> Pouring portable-ruby-2.6.3_2.yosemite.bottle.tar.gz
Update them from Software Update in System Preferences or run:
  softwareupdate --all --install --force

If that doesn't show you an update run:
  sudo rm -rf /Library/Developer/CommandLineTools
  sudo xcode-select --install

Alternatively, manually download them from:
  https://developer.apple.com/download/more/.

Error: An exception occurred within a child process:
  FormulaUnavailableError: No available formula with the name "/usr/local/opt/python@2/.brew/python@2.rb" 


In [34]:
%%bash
pip install sentencepiece
pip install wordcloud
pip uninstall tensorflow-hub
pip install tensorflow-hub==0.7.0

Found existing installation: tensorflow-hub 0.8.0
Uninstalling tensorflow-hub-0.8.0:
  Would remove:
    /Users/zhilinfang/anaconda3/bin/make_image_classifier
    /Users/zhilinfang/anaconda3/bin/make_nearest_neighbour_index
    /Users/zhilinfang/anaconda3/lib/python3.6/site-packages/tensorflow_hub-0.8.0.dist-info/*
    /Users/zhilinfang/anaconda3/lib/python3.6/site-packages/tensorflow_hub/*
Proceed (y/n)? Your response ('pip install tensorflow-hub==0.7.0') was not one of the expected responses: y, n
Proceed (y/n)? 

ERROR: Exception:
Traceback (most recent call last):
  File "/Users/zhilinfang/anaconda3/lib/python3.6/site-packages/pip/_internal/cli/base_command.py", line 186, in _main
    status = self.run(options, args)
  File "/Users/zhilinfang/anaconda3/lib/python3.6/site-packages/pip/_internal/commands/uninstall.py", line 79, in run
    auto_confirm=options.yes, verbose=self.verbosity > 0,
  File "/Users/zhilinfang/anaconda3/lib/python3.6/site-packages/pip/_internal/req/req_install.py", line 687, in uninstall
    uninstalled_pathset.remove(auto_confirm, verbose)
  File "/Users/zhilinfang/anaconda3/lib/python3.6/site-packages/pip/_internal/req/req_uninstall.py", line 388, in remove
    if auto_confirm or self._allowed_to_proceed(verbose):
  File "/Users/zhilinfang/anaconda3/lib/python3.6/site-packages/pip/_internal/req/req_uninstall.py", line 431, in _allowed_to_proceed
    return ask('Proceed (y/n)? ', ('y', 'n')) == 'y'
  File "/Users/zhilinfang/anaconda3/lib/python3.6/site-packages/pip/_inte

CalledProcessError: Command 'b'pip install sentencepiece\npip install wordcloud\npip uninstall tensorflow-hub\npip install tensorflow-hub==0.7.0\n'' returned non-zero exit status 2.

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import feature_extraction, linear_model, model_selection, preprocessing

In [33]:
import gc
import re
import string
import operator
from collections import defaultdict

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import matplotlib.pyplot as plt
import seaborn as sns

import tokenization
from wordcloud import STOPWORDS

from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from sklearn.metrics import precision_score, recall_score, f1_score

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.layers import Dense, Input, Dropout, GlobalAveragePooling1D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, Callback

SEED = 1627 #play with this!t5ffn fff                                                

ModuleNotFoundError: No module named 'tensorflow.python.training.tracking'

In [4]:
df_train = pd.read_csv('data/train.csv', dtype={'id': np.int16, 'target': np.int8})
df_test = pd.read_csv('data/test.csv', dtype={'id': np.int16})

print('Training Set Shape = {}'.format(df_train.shape))
print('Training Set Memory Usage = {:.2f} MB'.format(df_train.memory_usage().sum() / 1024**2))
print('Test Set Shape = {}'.format(df_test.shape))
print('Test Set Memory Usage = {:.2f} MB'.format(df_test.memory_usage().sum() / 1024**2))

Training Set Shape = (7613, 5)
Training Set Memory Usage = 0.20 MB
Test Set Shape = (3263, 4)
Test Set Memory Usage = 0.08 MB


In [6]:
df_train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


Filling missing values

1.1 Missing Values
Both training and test set have same ratio of missing values in keyword and location.

0.8% of keyword is missing in both training and test set
33% of location is missing in both training and test set
Since missing value ratios between training and test set are too close, they are most probably taken from the same sample. Missing values in those features are filled with no_keyword and no_location respectively.

In [7]:
missing_cols = ['keyword', 'location']

fig, axes = plt.subplots(ncols=2, figsize=(17, 4), dpi=100)

sns.barplot(x=df_train[missing_cols].isnull().sum().index, y=df_train[missing_cols].isnull().sum().values, ax=axes[0])
sns.barplot(x=df_test[missing_cols].isnull().sum().index, y=df_test[missing_cols].isnull().sum().values, ax=axes[1])

axes[0].set_ylabel('Missing Value Count', size=15, labelpad=20)
axes[0].tick_params(axis='x', labelsize=15)
axes[0].tick_params(axis='y', labelsize=15)
axes[1].tick_params(axis='x', labelsize=15)
axes[1].tick_params(axis='y', labelsize=15)

axes[0].set_title('Training Set', fontsize=13)
axes[1].set_title('Test Set', fontsize=13)

plt.show()

for df in [df_train, df_test]:
    for col in ['keyword', 'location']:
        df[col] = df[col].fillna(f'no_{col}')
    

NameError: name 'plt' is not defined

In [5]:
train_df[train_df["target"] == 0]["text"].values[1]

'I love fruits'

In [6]:
train_df[train_df["target"] == 1]["text"].values[1]

'Forest fire near La Ronge Sask. Canada'

In [7]:
count_vectorizer = feature_extraction.text.CountVectorizer()

## let's get counts for the first 5 tweets in the data
example_train_vectors = count_vectorizer.fit_transform(train_df["text"][0:5])

In [8]:
example_train_vectors

<5x54 sparse matrix of type '<class 'numpy.int64'>'
	with 61 stored elements in Compressed Sparse Row format>

In [11]:
print(example_train_vectors[0])

  (0, 34)	1
  (0, 12)	1
  (0, 5)	1
  (0, 49)	1
  (0, 39)	1
  (0, 29)	1
  (0, 50)	1
  (0, 13)	1
  (0, 25)	1
  (0, 4)	1
  (0, 18)	1
  (0, 52)	1
  (0, 3)	1


In [12]:
## we use .todense() here because these vectors are "sparse" (only non-zero elements are kept to save space)
print(example_train_vectors[0].todense().shape)
print(example_train_vectors[0].todense())

(1, 54)
[[0 0 0 1 1 1 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0
  0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 1 0]]


In [13]:
train_vectors = count_vectorizer.fit_transform(train_df["text"])

## note that we're NOT using .fit_transform() here. Using just .transform() makes sure
# that the tokens in the train vectors are the only ones mapped to the test vectors - 
# i.e. that the train and test vectors use the same set of tokens.
test_vectors = count_vectorizer.transform(test_df["text"])

In [14]:
## Our vectors are really big, so we want to push our model's weights
## toward 0 without completely discounting different words - ridge regression 
## is a good way to do this.
clf = linear_model.RidgeClassifier()